# 데이터 수집

## 전국 2023, 2024년 축제
* https://www.data.go.kr/data/15013104/standard.do
* 전국에서 개최되는 2023, 2024년 축제 이름, 축제 진행되는 날짜, 날씨에 따른 축제를 추천해주는 서비스 제공합니다.
* 제공기관 : 문화체육관광부
* 업데이트 주기 : 분기 (수정일:2023-07-13)

In [2]:
import folium
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import html
import datetime
from folium.plugins import MarkerCluster
import time
import seaborn as sns
from folium import Marker
import webbrowser
from matplotlib import font_manager,rc

font_location = "C:\Windows\Fonts\malgun.ttf"
font_name = font_manager.FontProperties(fname=font_location).get_name()
rc('font',family=font_name)

df_main=pd.read_csv('전국문화축제표준데이터.csv', encoding='cp949')
df_main.shape

(1397, 18)

## 초기 데이터 분석

In [3]:
df_main.head(2)

,축제명,개최장소,축제시작일자,축제종료일자,축제내용,주관기관,주최기관,후원기관,전화번호,홈페이지주소,관련정보,소재지도로명주소,소재지지번주소,위도,경도,데이터기준일자,제공기관코드,제공기관명
0,풍기인삼축제(2022 영주세계풍기인삼 엑스포),축제풍기인삼문화팝업공원 일원,2022-09-30,2022-10-23,"풍기기네스, 풍퀴즈 온더블록, 인삼향 풍기네, 슬기로운 풍기인삼 릴레이콘서트 등",(재)영주세계풍기인삼엑스포조직위원회,영주시,NaN,054-635-0020,http://www.ginsengfestival.co.kr/,NaN,경상북도 영주시 풍기읍 남원로 80,NaN,NaN,NaN,2022-09-28,B551011,한국관광공사
1,산청한방약초축제,산청IC축제광장+ 동의보감촌,2022-09-30,2022-10-11,"동의보감촌 허준순례길 구절초,한방촌거리 체험(대장간, 한의원, 약선음식 체험),동의...",산청한방약초축제위원회,산청군,산청군+산청군농협+한국수력원자력(주)산청양수발전소,055-970-6601,http://scherb.or.kr/,NaN,경상남도 산청군 금서면 동의보감로555번길 61,NaN,NaN,NaN,2022-09-28,B551011,한국관광공사


In [4]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1397 entries, 0 to 1396
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   축제명       1397 non-null   object 
 1   개최장소      1397 non-null   object 
 2   축제시작일자    1397 non-null   object 
 3   축제종료일자    1397 non-null   object 
 4   축제내용      1397 non-null   object 
 5   주관기관      1397 non-null   object 
 6   주최기관      1397 non-null   object 
 7   후원기관      689 non-null    object 
 8   전화번호      1257 non-null   object 
 9   홈페이지주소    918 non-null    object 
 10  관련정보      465 non-null    object 
 11  소재지도로명주소  1262 non-null   object 
 12  소재지지번주소   1095 non-null   object 
 13  위도        1181 non-null   float64
 14  경도        1181 non-null   float64
 15  데이터기준일자   1397 non-null   object 
 16  제공기관코드    1397 non-null   object 
 17  제공기관명     1397 non-null   object 
dtypes: float64(2), object(16)
memory usage: 196.6+ KB


## 데이터 클렌징 (지도 분석데이터 선정)

### 필요없는 열 삭제

In [19]:
df_main.drop(['데이터기준일자','제공기관코드','제공기관명'], axis=1, inplace=True)
df_main

,축제명,개최장소,축제시작일자,축제종료일자,축제내용,주최기관,전화번호,홈페이지주소,관련정보,소재지도로명주소,소재지지번주소,위도,경도
0,풍기인삼축제(2022 영주세계풍기인삼 엑스포),축제풍기인삼문화팝업공원 일원,2022-09-30,2022-10-23,"풍기기네스, 풍퀴즈 온더블록, 인삼향 풍기네, 슬기로운 풍기인삼 릴레이콘서트 등",영주시,054-635-0020,http://www.ginsengfestival.co.kr/,NaN,경상북도 영주시 풍기읍 남원로 80,NaN,NaN,NaN
1,산청한방약초축제,산청IC축제광장+ 동의보감촌,2022-09-30,2022-10-11,"동의보감촌 허준순례길 구절초,한방촌거리 체험(대장간, 한의원, 약선음식 체험),동의...",산청군,055-970-6601,http://scherb.or.kr/,NaN,경상남도 산청군 금서면 동의보감로555번길 61,NaN,NaN,NaN
2,양양송이축제,강원도 양양군 전통시장 및 남대천 일원,2022-09-30,2022-10-02,"송이 보물찾기,송이돌이 만들기, 목공예체험, 버블쇼 공연, 송이볼링과 송이 과녁컬링",양양군,033-671-3803,http://yymsfestival.com/index.php,NaN,NaN,강원도 양양군 양양읍 남문리 226-2,NaN,NaN
3,울산옹기축제,온라인 축제,2022-09-30,2022-10-03,"국내 유일의 옹기특화 축제, 온라인비대면축제",울산광역시 울주군청 관광과,052-204-0355,https://www.onggi.or.kr/,NaN,울산광역시 울주군 온양읍 외고산2길 2-13,울산광역시 울주군 온양읍 고산리 512-4,35.432991,129.278447
4,금산인삼축제,금산인삼관 광장 및 인삼약초시장 일원,2022-09-30,2022-10-23,ㅇ인삼캐기 체험+인삼약초체험+생활건강 체험 등 건강 체험관 운영+국제인삼교역전+인삼...,충청남도 금산군,041-750-4144,http://www.insamfestival.co.kr/,NaN,충청남도 금산군 금산읍 인삼광장로 30,충청남도 금산군 금산읍 신대리 392,36.100026,127.500730
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1392,힐링 개그 콘서트,꽃바위문화관 3층 다목적공연장,2021-08-26,2021-08-26,개그x싱어롱,꽃바위문화관,052-209-4330,http://www.donggu.ulsan.kr/edu/menu_06/03.jsp,NaN,울산광역시 동구 문현3길 6,울산광역시 동구 방어동 1120-1,35.487058,129.411263
1393,오페라 팝페라 뽕페라,꽃바위문화관 3층 다목적공연장,2021-09-30,2021-09-30,콘서트,꽃바위문화관,052-209-4330,http://www.donggu.ulsan.kr/edu/menu_06/03.jsp,NaN,울산광역시 동구 문현3길 6,울산광역시 동구 방어동 1120-1,35.487058,129.411263
1394,박강수콘서트,꽃바위문화관 3층 다목적공연장,2021-10-21,2021-10-21,콘서트,꽃바위문화관,052-209-4330,http://www.donggu.ulsan.kr/edu/menu_06/03.jsp,NaN,울산광역시 동구 문현3길 6,울산광역시 동구 방어동 1120-1,35.487058,129.411263
1395,대왕암 해맞이 축제,대왕암,2023-01-01,2023-01-01,해맞이,동울산청년회의소,NaN,NaN,NaN,NaN,울산광역시 동구 일산동 산907,NaN,NaN


## 위도, 경도 결측값 if문으로 넘어가기

In [26]:
df_main['위도']

0             NaN
1             NaN
2             NaN
3       35.432991
4       36.100026
          ...    
1392    35.487058
1393    35.487058
1394    35.487058
1395          NaN
1396    35.487058
Name: 위도, Length: 1397, dtype: float64

In [27]:
df_main['위도'] = df_main['위도'].fillna(0)

In [28]:
df_main['위도']

0        0.000000
1        0.000000
2        0.000000
3       35.432991
4       36.100026
          ...    
1392    35.487058
1393    35.487058
1394    35.487058
1395     0.000000
1396    35.487058
Name: 위도, Length: 1397, dtype: float64

In [30]:
df_main['경도'] = df_main['경도'].fillna(0)

In [31]:
df_main['경도']

0         0.000000
1         0.000000
2         0.000000
3       129.278447
4       127.500730
           ...    
1392    129.411263
1393    129.411263
1394    129.411263
1395      0.000000
1396    129.411263
Name: 경도, Length: 1397, dtype: float64

## 주소지 하나로 합치기

## 2023,2024년 진행되는 축제만 남기기

In [33]:
df_main['축제시작일자']=pd.to_datetime(df_main['축제시작일자'])
df_main['축제종료일자']=pd.to_datetime(df_main['축제종료일자'])
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1397 entries, 0 to 1396
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   축제명       1397 non-null   object        
 1   개최장소      1397 non-null   object        
 2   축제시작일자    1397 non-null   datetime64[ns]
 3   축제종료일자    1397 non-null   datetime64[ns]
 4   축제내용      1397 non-null   object        
 5   주최기관      1397 non-null   object        
 6   전화번호      1257 non-null   object        
 7   홈페이지주소    918 non-null    object        
 8   관련정보      465 non-null    object        
 9   소재지도로명주소  1262 non-null   object        
 10  소재지지번주소   1095 non-null   object        
 11  위도        1397 non-null   float64       
 12  경도        1397 non-null   float64       
dtypes: datetime64[ns](2), float64(2), object(9)
memory usage: 142.0+ KB


In [37]:
df_main23 = df_main[df_main['축제시작일자']>= pd.to_datetime('2023')]
df_main23.to_csv('축제2023.csv', mode='w', encoding='cp949')
df_main23.info()


<class 'pandas.core.frame.DataFrame'>
Index: 410 entries, 93 to 1396
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   축제명       410 non-null    object        
 1   개최장소      410 non-null    object        
 2   축제시작일자    410 non-null    datetime64[ns]
 3   축제종료일자    410 non-null    datetime64[ns]
 4   축제내용      410 non-null    object        
 5   주최기관      410 non-null    object        
 6   전화번호      378 non-null    object        
 7   홈페이지주소    245 non-null    object        
 8   관련정보      49 non-null     object        
 9   소재지도로명주소  341 non-null    object        
 10  소재지지번주소   329 non-null    object        
 11  위도        410 non-null    float64       
 12  경도        410 non-null    float64       
dtypes: datetime64[ns](2), float64(2), object(9)
memory usage: 44.8+ KB


## 전국 지역축제 : 맵에 전체 지도데이터 표시

In [42]:
# 위도
latitude = 35.22
# 경도
longitude = 127.77

In [54]:
m = folium.Map(location=[latitude, longitude],
               zoom_start=7,                
               width=750, 
               height=500
              )
m

In [55]:
df_addr = df_main[['축제명','개최장소','축제내용','위도','경도','전화번호']]
df_addr.head(2)

,축제명,개최장소,축제내용,위도,경도,전화번호
0,풍기인삼축제(2022 영주세계풍기인삼 엑스포),축제풍기인삼문화팝업공원 일원,"풍기기네스, 풍퀴즈 온더블록, 인삼향 풍기네, 슬기로운 풍기인삼 릴레이콘서트 등",0.0,0.0,054-635-0020
1,산청한방약초축제,산청IC축제광장+ 동의보감촌,"동의보감촌 허준순례길 구절초,한방촌거리 체험(대장간, 한의원, 약선음식 체험),동의...",0.0,0.0,055-970-6601


In [70]:
def all_map():
    

    m = folium.Map(location=[35.22, 127.77],zoom_start=7, popup='',width=750, height=750,)

    for name, lat, lng, m_addr, m_info in zip(df_main23['축제명'],df_main23['위도'],df_main23['경도'],df_main23['개최장소'],df_main23['전화번호']):
        Marker(location =[lat,lng],
               zoom_start=7, 
               
               tooltip=name,
               icon=folium.Icon(color='green',icon='bookmark')
          
              ).add_to(m)
    return m

In [71]:
all_map()

## 클러스터

In [74]:
def all_cl():
    # 위도
    latitude = 35.22
    # 경도
    longitude = 127.77

    m4 = folium.Map(location=[latitude, longitude],
                   zoom_start=7, 
                   width=1000, 
                   height=600
                  )

    coords = df_main[['위도','경도']]
    marker_cluster = MarkerCluster().add_to(m4)

    for lat, long in zip(coords['위도'], coords['경도']):
        folium.Marker([lat, long], icon = folium.Icon(color="green")).add_to(marker_cluster)
    return m4


In [76]:
all_cl()

# 검색 조건 (축제명 / 개최장소)

## 함수

### 축제명으로 검색

In [77]:
# search 함수 있으면 True 반환
def search(word,x):
#     p="\D+["+word+"]\D+"  # 실패한 흔적
    a=str(x)
    rs=a.find(word)
    if rs == -1:
        return False
    else:
        return True


In [78]:
## 두번 쉬프트 엔터 누르면 아나콘다 재시작해야함.
#I_name = input('소재지를 입력하세요 :  ')

l_name = '치악산찰옥수수축제' # 테스트

In [79]:
list_lo=df_main[df_main['개최장소'].apply(lambda x: search(l_name,x))]
list_lo

,축제명,개최장소,축제시작일자,축제종료일자,축제내용,주최기관,전화번호,홈페이지주소,관련정보,소재지도로명주소,소재지지번주소,위도,경도


In [80]:
# name_list= input('개최장소를 입력하세요 : ')
m_name='법천소공원'

In [81]:
name_list=df_main[df_main['축제명']==m_name]
name_list

,축제명,개최장소,축제시작일자,축제종료일자,축제내용,주최기관,전화번호,홈페이지주소,관련정보,소재지도로명주소,소재지지번주소,위도,경도


## 지역검색

In [15]:
df_kang = df_main[df_main['명산_소재지'].str.contains('강원')] 

In [16]:
# 위도
latitude = 35.904946
    # 경도
longitude = 128.111104

m2 = folium.Map(location=[latitude, longitude],
               zoom_start=8, 
               width=750, 
               height=500
              )

In [ ]:
def go_map(x):
    m2=folium.Map(location=[latitude, longitude],
                zoom_start=7, 
                width=900, 
                height=600
              )
    for name, lat, lng, m_num  in zip(x['명산_이름'],x['X좌표'],x['Y좌표'],x['명산_소재지']):
        Marker(location =[lat,lng],
               popup=m_num,
               tooltip=name,
               icon=folium.Icon(color='blue',icon='bookmark')
               ).add_to(m2)
    return m2

In [ ]:
go_map(df_kang)

## 명산(전국) 지도데이터 - 클러스터

In [17]:
def all_cl():
    # 위도
    latitude = 35.904946
    # 경도
    longitude = 128.111104

    m4 = folium.Map(location=[latitude, longitude],
                   zoom_start=7, 
                   width=1000, 
                   height=600
                  )

    coords = df_main[['X좌표','Y좌표']]
    marker_cluster = MarkerCluster().add_to(m4)

    for lat, long in zip(coords['X좌표'], coords['Y좌표']):
        folium.Marker([lat, long], icon = folium.Icon(color="green")).add_to(marker_cluster)
    return m4


In [18]:
all_cl()

# 반복 서비스 프로그램

###아래 실행시 주의

In [19]:
a=True
while a:
    print("""
    주의사항 쉬프트 엔터를 두번 누르면 고장남
    1. 산이름 검색
    2. 지역이름검색
    3. 전체맵보기
    4. 전체클러스
    5. 종료
    
    """)

    a=input('명령어 입력:  ')
    print()
    if a=='종료' or a=='5':
        a=False
        
    if a=='산이름 검색' or a=='1':
        m_name=input('산이름 입력:  ')
        print(m_name + '을 입력하셨습니다.')
#         print(df_main[df_main['명산_이름']==m_name])
        x=go_map(df_main[df_main['명산_이름']==m_name])
        x.save("index.html")
        webbrowser.open_new_tab('index.html')
#         x.show_in_browser
#         print(df_main[df_main['명산_이름']==m_name])
    if a=='지역이름검색' or a =='2':
        l_name=input('지역 이름:  ')

        x=go_map(df_main[df_main['명산_소재지'].apply(lambda x: search(l_name,x))])
        x.save("index.html")
        webbrowser.open_new_tab('index.html')
    if a=='전체맵보기' or a=='3':
        x=all_map()
        x.save("index.html")
        webbrowser.open_new_tab('index.html')
    if a=='전체클러스터' or a =='4':
        x=all_cl()
        x.save("index.html")
        webbrowser.open_new_tab('index.html')



    주의사항 쉬프트 엔터를 두번 누르면 고장남
    1. 산이름 검색
    2. 지역이름검색
    3. 전체맵보기
    4. 전체클러스
    5. 종료
    
    
명령어 입력:  



In [ ]:
# m2.show_in_browser

In [20]:
webbrowser.open_new_tab('index.html')

True

In [21]:

# HTML 이스케이프 문자열을 원래의 HTML 문자로 변환
decoded_data = html.unescape(data)

# 변환된 HTML 코드 출력
print(decoded_data)

NameError: name 'data' is not defined

In [22]:
df_main['산행코스'].apply(lambda x: print(html.unescape(x)))

<table class="tbl mb_10">				<caption style="overflow:hidden; font-size:0; line-height:0; text-indent:-9999px;">가리산 산행코스</caption>				<colgroup>					<col style="width:20%;">					<col style="width:60%;">					<col style="width:20%;">				</colgroup>				<thead>					<tr>						<th>구분</th>						<th>구간</th>						<th>소요시간</th>					</tr>				</thead>				<tbody>							<tr>						<td>추천코스</td>						<td class="left">가리산자연휴양림  관리사무소 ① - 삼거리 ② - 가삽고개·북동능선 ③ - 정상 ④<br> - 샘터·남릉 등산안내도 ⑤ - 무쇠말재 ⑥ - 삼거리 ② - 휴양림 주차장 ① </td>						<td>약 3시간 35분</td>					</tr>					<tr>						<td>기타코스1</td>						<td class="left">가리산자연휴양림 ① - 삼거리 ② - 가삽고개 ③ - 소양호 물로리 ⑦</td>						<td>약 4시간 30분</td>					</tr>					<tr>						<td>기타코스2</td>						<td class="left">가리산자연휴양림 ① - 가삽고개 ③ - 정상 ④ - 가리산교 - 철정리 ⑧</td>						<td>약 4시간 30분</td>					</tr>				</tbody>			</table>
<table class="tbl mb_10">				<caption style="overflow:hidden; font-size:0; line-height:0; text-indent:-9999px;">가리왕산 산행코스</caption>				<colgroup>					<col style="width:20%;">

0     None
1     None
2     None
3     None
4     None
      ... 
95    None
96    None
97    None
98    None
99    None
Name: 산행코스, Length: 100, dtype: object